# River pollution problem (all objectives minimized)

In [1]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

from desdeo_problem.problem import DiscreteDataProblem
from desdeo_tools.scalarization import SimpleASF, DiscreteScalarizer, StomASF, PointMethodASF
from desdeo_tools.solver import DiscreteMinimizer
from shapley_values.explanations import *
from shapley_values.utilities import *
from IPython.core.display import display, HTML


# Load and setup problem
df = pd.read_csv("../data/river_pollution_10178.csv")
pareto_front = df.to_numpy()

ideal = np.min(pareto_front[:, 0:5], axis=0)
nadir = np.max(pareto_front[:, 0:5], axis=0)

objective_names = ["f_1", "f_2", "f_3", "f_4", "f_5"]
n_objectives = len(objective_names)

problem = DiscreteDataProblem(df, ["x_1", "x_2"], objective_names, nadir, ideal)

# asf = SimpleASF(np.array([1,1,1,1,1]))
asf = StomASF(ideal)
# asf = PointMethodASF(nadir, ideal)

missing_data = shap.sample(pareto_front[:, 0:n_objectives], nsamples=200)

# need to know reference point before generating missing data
bb = generate_black_box(problem, asf)
explainer = shap.KernelExplainer(bb, missing_data)

normalizer = Normalizer(ideal, nadir)

def plot(result, objective_names=objective_names, ymin=0, ymax=1):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_ylim(ymin, ymax)
    ax.bar(objective_names, np.squeeze(result)) 
    plt.grid()
    plt.show()

print(f"Ideal: {ideal}")
print(f"Nadir: {nadir}")
print(np.abs(nadir - ideal))

c:\Users\Giomara\Documents\Projects\Tradeoff_Analysis\vshap\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Giomara\AppData\Local\Temp\ipykernel_2532\3418348890.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


Ideal: [-6.34000000e+00 -3.44487179e+00 -7.50000000e+00  0.00000000e+00
  1.71409445e-03]
Nadir: [-4.751      -2.85595261 -0.32111111  9.70666667  0.35      ]
[1.589      0.58891919 7.17888889 9.70666667 0.34828591]


## 1st iteration

In [15]:
ref_point = np.array([-4.751,-2.85595261,-0.32111111,9.70666667,0.35])

result = bb(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))

print(f"Reference point: {ref_point}")
print(f"Result: {result}")
#plot(result, ymin=np.min(ideal))

def print_matrix(matrix):
    print("[")
    for row in matrix:
        print(f"[{', '.join(map(str, row))}],")
    print("]")
print_matrix(shap_values)

Reference point: [-4.751      -2.85595261 -0.32111111  9.70666667  0.35      ]
Result: [[-5.99961942 -3.10799147 -6.27755692  1.66335202  0.20091479]]
[
[0.08232780797566096, 0.019520080927165628, -0.7606810780763054, -0.36856838513740653, -0.029194050578624377],
[0.10336844293920648, 0.10279253251017428, -1.0070466704760204, -1.7995318886367055, -0.047990225112473216],
[-0.34946214645018153, 0.007054147576319551, 0.9990153173486036, -0.33996460044277427, -0.003171241562519754],
[0.034356288886672415, -0.03510515324007928, -0.34902695771141046, 0.3695948313466284, -0.009525827062155493],
[-0.06382402295776021, -0.019548945216339653, 0.349998163770189, 0.17726304449356967, 0.02568554563106086],
]


## Selecting the objective to be improved #1

In [16]:
to_improve = 0
suggestion_str, _, _, _ = how_to_improve_objective_i(shap_values, to_improve, ref_point, result)

def print_all_explanations(shap_values, ref_point, result):
    suggestions = []
    for i in range(0,5):
        suggestion_str, _, _, _ = how_to_improve_objective_i(shap_values, i, ref_point, result)
        suggestions.append(suggestion_str)
    return suggestions

print(print_all_explanations(shap_values, ref_point, result))

['Explanation: Each objective value in the solution had a bettern value when compared to the reference point. The reference point was pessimistic. The component Objective 2 in the refence point had the least improving effect on the objective Objective 1 in the solution.\nSuggestion: Try improving the component Objective 1 and impairing the component Objective 2.', 'Explanation: Each objective value in the solution had a bettern value when compared to the reference point. The reference point was pessimistic. The component Objective 1 in the refence point had the least improving effect on the objective Objective 2 in the solution.\nSuggestion: Try improving the component Objective 2 and impairing the component Objective 1.', 'Explanation: Each objective value in the solution had a better value when compared to the reference point.The reference point was pessimistic. The component Objective 3 in the reference point had the least improving effect on objective Objective 3 in the solution. T

## 2nd iteration

In [18]:
ref_point = np.array([-6, -3.2, -6.0, 0.1, 0.1])

result = bb(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))



print(f"Reference point: {ref_point}")
print(f"Result: {result}")
print_matrix(shap_values)

Reference point: [-6.  -3.2 -6.   0.1  0.1]
Result: [[-6.00345962 -2.91928927 -6.26229508  0.21316082  0.20174433]]
[
[0.004280237176539252, 0.007413895737360971, -0.483937992334114, -0.10188277289418518, -0.017141842065165388],
[0.0246457504475303, 0.03662135153711577, -0.7283604491506108, -0.6702866047828461, -0.014716242748077911],
[-0.2070728223028497, -0.002142501614164864, 0.21669451803035128, -0.05452464752482899, 0.0024593705263770767],
[-0.09111362491129119, 0.17206395179244688, 0.8286678891717021, -1.8660718850579148, 0.020434617057692808],
[0.0721866328121003, 0.04945816128211339, -0.5855433498248166, -0.7186322874337727, -0.054402162828639064],
]


## Selecting the objective to be improved #2

In [19]:
# to_improve = 3
suggestion_str, _, _, _ = how_to_improve_objective_i(shap_values, to_improve, ref_point, result)

print(print_all_explanations(shap_values, ref_point, result))

['Explanation: The objective Objective 1 was most improved in the solution by the component Objective 3 and most impaired by the component Objective 2 in the reference point.\nSuggestion: Try improving the component Objective 1 and impairing the component Objective 2.', 'Explanation: The objective Objective 2 was most impaired in the solution by its component in the reference point. The component Objective 1 had the second most impairing effect on the objective Objective 2.\nSuggestion: Try improving the component Objective 2 and impairing the component Objective 1.', 'Explanation: The objective Objective 3 was most impaired in the solution by its component in the reference point. The component Objective 5 had the second most impairing effect on the objective Objective 3.\nSuggestion: Try improving the component Objective 3 and impairing the component Objective 5.', 'Explanation: The objective Objective 4 was most improved in the solution by its component in the reference point. The co

## 3rd teration

In [22]:
ref_point = np.array([-6, -3.2, -6.0, 1, 0.1])

result = bb(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))

print(f"Reference point: {ref_point}")
print(f"Result: {result}")
print_matrix(shap_values)

Reference point: [-6.  -3.2 -6.   1.   0.1]
Result: [[-5.90002751 -3.05750649 -6.59663711  1.20663092  0.15617952]]
[
[0.01000648425547853, 0.0045294520552264655, -0.5289871098665174, -0.08632736101265501, -0.021020476026540252],
[0.024443385017591825, 0.03020854037937561, -0.391084012490849, -0.7576582888125346, -0.014208353484995712],
[-0.1938555864559639, -0.0019406278455032662, 0.11262071484170526, -0.05389655963242717, -0.0060082786506903355],
[-0.018503223442102545, 0.03459217477624763, 0.23118523644384936, -0.726184332950595, -0.008826478556888917],
[0.08426722191781977, 0.057808101797993755, -0.5105562413833893, -0.7938615588022513, -0.058867478656227955],
]


## Selecting the objective to be improved #3

In [24]:
print(print_all_explanations(shap_values, ref_point, result))

['Explanation: The objective Objective 1 was most improved in the solution by its component in the reference point. The component Objective 2 had the most impairing effect of objective Objective 1.\nSuggestion: Try improving the component Objective 1 and impairing the component Objective 2.', 'Explanation: The objective Objective 2 was most improved in the solution by its component in the reference point. The component Objective 1 had the most impairing effect of objective Objective 2.\nSuggestion: Try improving the component Objective 2 and impairing the component Objective 1.', 'Explanation: None of the objectives in the reference point had an impairing effect on objective Objective 3 in the solution. Objective Objective 2 in the reference point had the least improving effect on objective Objective 3 in the solution.\nSuggestion: Try improving the component Objective 3 and impairing the component Objective 2.', 'Explanation: The objective Objective 4 was most improved in the solution

##  4th iteration

In [25]:
ref_point = np.array([-6, -3.2, -5.5, 0.8, 0.12])

result = bb(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))

print(f"Reference point: {ref_point}")
print(f"Result: {result}")
print_matrix(shap_values)

Reference point: [-6.   -3.2  -5.5   0.8   0.12]
Result: [[-5.95206821 -3.06336352 -6.44639728  1.24607038  0.17910494]]
[
[0.0029064904587753604, 0.003492084257043458, -0.5062318788811963, -0.06792903178744268, -0.018690773514580275],
[0.02724277414151736, 0.027714994673892965, -0.5586714937708296, -0.7307785275875508, -0.013596774905201161],
[-0.21768072958799214, -0.002238640942598349, 0.2739868664273693, -0.08089484042540025, -4.0484318092023425e-05],
[-0.028346964975635585, 0.048877583157491314, 0.3622676731796052, -0.8807778475899458, -0.006404087852124278],
[0.07019600798439357, 0.04149458641265302, -0.5079327490667939, -0.6181083900974713, -0.047273524841836626],
]


## Selecting the objective to be improved #5

In [26]:
print_all_explanations(shap_values,ref_point,result)

['Explanation: The objective Objective 1 was most improved in the solution by the component Objective 3 and most impaired by the component Objective 2 in the reference point.\nSuggestion: Try improving the component Objective 1 and impairing the component Objective 2.',
 'Explanation: The objective Objective 2 was most impaired in the solution by its component in the reference point. The component Objective 1 had the second most impairing effect on the objective Objective 2.\nSuggestion: Try improving the component Objective 2 and impairing the component Objective 1.',
 'Explanation: The objective Objective 3 was most impaired in the solution by its component in the reference point. The component Objective 5 had the second most impairing effect on the objective Objective 3.\nSuggestion: Try improving the component Objective 3 and impairing the component Objective 5.',
 'Explanation: The objective Objective 4 was most improved in the solution by its component in the reference point. The

##  5th iteration

In [27]:
ref_point = np.array([-6, -3.2, -5.5, 0.8, 0.12])

result = bb(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))

print(f"Reference point: {ref_point}")
print(f"Result: {result}")
print_matrix(shap_values)

Reference point: [-6.   -3.2  -5.5   0.8   0.12]
Result: [[-5.95206821 -3.06336352 -6.44639728  1.24607038  0.17910494]]
[
[0.0029064904587753604, 0.003492084257043458, -0.5062318788811963, -0.06792903178744268, -0.018690773514580275],
[0.02724277414151736, 0.027714994673892965, -0.5586714937708296, -0.7307785275875508, -0.013596774905201161],
[-0.21768072958799214, -0.002238640942598349, 0.2739868664273693, -0.08089484042540025, -4.0484318092023425e-05],
[-0.028346964975635585, 0.048877583157491314, 0.3622676731796052, -0.8807778475899458, -0.006404087852124278],
[0.07019600798439357, 0.04149458641265302, -0.5079327490667939, -0.6181083900974713, -0.047273524841836626],
]


## Selecting the objective to be improved #5

In [28]:

print_all_explanations(shap_values, ref_point, result)



['Explanation: The objective Objective 1 was most improved in the solution by the component Objective 3 and most impaired by the component Objective 2 in the reference point.\nSuggestion: Try improving the component Objective 1 and impairing the component Objective 2.',
 'Explanation: The objective Objective 2 was most impaired in the solution by its component in the reference point. The component Objective 1 had the second most impairing effect on the objective Objective 2.\nSuggestion: Try improving the component Objective 2 and impairing the component Objective 1.',
 'Explanation: The objective Objective 3 was most impaired in the solution by its component in the reference point. The component Objective 5 had the second most impairing effect on the objective Objective 3.\nSuggestion: Try improving the component Objective 3 and impairing the component Objective 5.',
 'Explanation: The objective Objective 4 was most improved in the solution by its component in the reference point. The

##  6th iteration

In [29]:
ref_point = np.array([-6, -3.2, -5.5, 0.8, 0.12])

result = bb(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))

print(f"Reference point: {ref_point}")
print(f"Result: {result}")
print_matrix(shap_values)
print_all_explanations(shap_values,ref_point,result)

Reference point: [-6.   -3.2  -5.5   0.8   0.12]
Result: [[-5.95206821 -3.06336352 -6.44639728  1.24607038  0.17910494]]
[
[0.0029064904587753604, 0.003492084257043458, -0.5062318788811963, -0.06792903178744268, -0.018690773514580275],
[0.02724277414151736, 0.027714994673892965, -0.5586714937708296, -0.7307785275875508, -0.013596774905201161],
[-0.21768072958799214, -0.002238640942598349, 0.2739868664273693, -0.08089484042540025, -4.0484318092023425e-05],
[-0.028346964975635585, 0.048877583157491314, 0.3622676731796052, -0.8807778475899458, -0.006404087852124278],
[0.07019600798439357, 0.04149458641265302, -0.5079327490667939, -0.6181083900974713, -0.047273524841836626],
]


['Explanation: The objective Objective 1 was most improved in the solution by the component Objective 3 and most impaired by the component Objective 2 in the reference point.\nSuggestion: Try improving the component Objective 1 and impairing the component Objective 2.',
 'Explanation: The objective Objective 2 was most impaired in the solution by its component in the reference point. The component Objective 1 had the second most impairing effect on the objective Objective 2.\nSuggestion: Try improving the component Objective 2 and impairing the component Objective 1.',
 'Explanation: The objective Objective 3 was most impaired in the solution by its component in the reference point. The component Objective 5 had the second most impairing effect on the objective Objective 3.\nSuggestion: Try improving the component Objective 3 and impairing the component Objective 5.',
 'Explanation: The objective Objective 4 was most improved in the solution by its component in the reference point. The